# T S. Vishnu [1], A. Bhattacharya [2]
#### 1 - Junior Undergrad, 2 - Senior Undergrad
### Indian Institute of Technology

In [1]:
from collections import Counter
from string import punctuation
import numpy as np

In [2]:
text_f = open('imdb-data/reviews.txt','r')
reviews = text_f.read()
labels_f = open('imdb-data/labels.txt','r')
labels = labels_f.read()

In [3]:
print(reviews[:1000])
print(labels[:16])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

In [4]:
reviews = reviews.lower()
text = ''.join([c for c in reviews if c not in punctuation])

In [5]:
reviews_split = text.split('\n')
text = ' '.join(reviews_split)

# create a list of words
words = text.split()

In [6]:
print(len(reviews_split))
labels = labels.split("\n")
print(len(labels))

25001
25001


In [7]:
print(text[:100])
print(words[10])

bromwell high is a cartoon comedy  it ran at the same time as some other programs about school life 
same


In [8]:

## Build a dictionary that maps words to integers 
counts = Counter(words)

In [9]:
print(len(counts))
print(len(words))

74072
6020196


In [10]:
vocab = sorted(counts, key=counts.get, reverse=True)
print(len(vocab))

74072


In [11]:
vocab_to_int = {word: i for i, word in enumerate(vocab, 1)}

In [12]:
print(len(vocab_to_int))

74072


In [13]:
#print(len(encoded_reviews))

In [ ]:
encoded_reviews = []
[encoded_reviews.append([vocab_to_int[word] for word in r.split()]) for r in reviews_split]

In [15]:
# check if the encoded reviews length is the same as a normal review
#[print(len(reviews_split[i].split()), len(encoded_reviews[i])) for i in range(len(reviews_split))]

In [16]:
encoded_labels = np.array([1 if label == 'positive' else 0 for label in labels])
print(labels[:1])

['positive']


In [17]:
# outlier review stats
review_lens = Counter([len(x) for x in encoded_reviews])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum review length: 2514


In [18]:
## remove any reviews/labels with zero length from the reviews_ints list.
# get indices of any reviews with length 0 
non_zero_idx = [ii for ii, review in enumerate(encoded_reviews) if len(review) != 0]
# remove 0-length reviews and their labels 
reviews_ints = [encoded_reviews[ii] for ii in non_zero_idx] 
encoded_labels = np.array([encoded_labels[ii] for ii in non_zero_idx])  
print('Number of reviews after removing outliers: ', len(reviews_ints))

Number of reviews after removing outliers:  25000


In [19]:
print(len(encoded_labels))

25000


In [20]:
def pad_features(reviews_ints, seq_length):
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    return features

In [21]:
seq_length = 200

features = pad_features(reviews_ints, seq_length=seq_length)

In [22]:
[print(features[i][:200], encoded_labels[i]) for i in range(3)]

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
 21025   308     6     3  1050   207     8  2138    32     1   171    57
    15    49    81  5785    44   382   110   140    15  5194    60   154
     9     1  4975  5852   475    71     5   260    12 21025   308    13
  1978     6    74  2395     5   613    73     6  5194     1 24103     5
  1983 10166     1  5786  1499    36    51    66   204   145    67  1199
  5194 19869     1 37442     4     1   221   883    31  2988    71     4
     1  5787    10   686     2    67  1499    54    10   216     1   383
     9    62     3  1406  3686   783     5  3483   180     1   382    10
  1212 13583    32   308     3   349   341  2913   

[None, None, None]

In [23]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2500, 200)


In [24]:
print(len(train_x), len(train_y))

20000 20000


In [25]:
print(len(test_x), len(test_y))

2500 2500


In [26]:
print(len(val_x), len(val_y))

2500 2500


In [27]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [28]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 200])
Sample input: 
 tensor([[   0,    0,    0,  ...,   52,  724,   18],
        [  30,    4,    1,  ..., 1374,   20,   34],
        [   0,    0,    0,  ...,   12,   16, 1544],
        ...,
        [   0,    0,    0,  ...,   23,   29,  672],
        [ 988, 2035, 4214,  ...,    3,   74, 6914],
        [   0,    0,    0,  ...,    7,    1, 7819]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
        0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
        1, 0])


In [45]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

No GPU available, training on CPU.


In [29]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [30]:
net = SentimentRNN(74072, 1, 200, 128, 2)

In [31]:
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [32]:
import timeit

In [53]:
# training params

epochs = 4 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 10
clip=5 # gradient clipping

    


# move model to GPU, if available
if(train_on_gpu):
    net.cuda()
    
net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    print(timeit.default_timer())
    for inputs, labels in train_loader:
        counter += 1
        #print(counter)
        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            print(counter)
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

2214.126296148
10
Epoch: 1/4... Step: 10... Loss: 0.552676... Val Loss: 0.566647
20
Epoch: 1/4... Step: 20... Loss: 0.478660... Val Loss: 0.584911
30
Epoch: 1/4... Step: 30... Loss: 0.535999... Val Loss: 0.575509
40
Epoch: 1/4... Step: 40... Loss: 0.431518... Val Loss: 0.572853
50
Epoch: 1/4... Step: 50... Loss: 0.654851... Val Loss: 0.561881
60
Epoch: 1/4... Step: 60... Loss: 0.443313... Val Loss: 0.602107
70
Epoch: 1/4... Step: 70... Loss: 0.619176... Val Loss: 0.572406
80
Epoch: 1/4... Step: 80... Loss: 0.504361... Val Loss: 0.552677
90
Epoch: 1/4... Step: 90... Loss: 0.450999... Val Loss: 0.546876
100
Epoch: 1/4... Step: 100... Loss: 0.785503... Val Loss: 0.585338
110
Epoch: 1/4... Step: 110... Loss: 0.584925... Val Loss: 0.533098
120
Epoch: 1/4... Step: 120... Loss: 0.539047... Val Loss: 0.577684
130
Epoch: 1/4... Step: 130... Loss: 0.512821... Val Loss: 0.581343
140
Epoch: 1/4... Step: 140... Loss: 0.529345... Val Loss: 0.585661
150
Epoch: 1/4... Step: 150... Loss: 0.389597... Va

1210
Epoch: 4/4... Step: 1210... Loss: 0.307710... Val Loss: 0.538702
1220
Epoch: 4/4... Step: 1220... Loss: 0.238222... Val Loss: 0.500253
1230
Epoch: 4/4... Step: 1230... Loss: 0.298452... Val Loss: 0.536204
1240
Epoch: 4/4... Step: 1240... Loss: 0.176908... Val Loss: 0.520016
1250
Epoch: 4/4... Step: 1250... Loss: 0.144472... Val Loss: 0.507130
1260
Epoch: 4/4... Step: 1260... Loss: 0.295805... Val Loss: 0.560244
1270
Epoch: 4/4... Step: 1270... Loss: 0.305527... Val Loss: 0.499520
1280
Epoch: 4/4... Step: 1280... Loss: 0.165900... Val Loss: 0.525920
1290
Epoch: 4/4... Step: 1290... Loss: 0.178712... Val Loss: 0.530073
1300
Epoch: 4/4... Step: 1300... Loss: 0.170125... Val Loss: 0.501474
1310
Epoch: 4/4... Step: 1310... Loss: 0.218819... Val Loss: 0.507185
1320
Epoch: 4/4... Step: 1320... Loss: 0.229444... Val Loss: 0.547995
1330
Epoch: 4/4... Step: 1330... Loss: 0.397476... Val Loss: 0.542285
1340
Epoch: 4/4... Step: 1340... Loss: 0.140559... Val Loss: 0.493368
1350
Epoch: 4/4... S

In [55]:
test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in train_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    #if(train_on_gpu):
        #inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Train loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(train_loader.dataset)
print("Train accuracy: {:.3f}".format(test_acc))

Train loss: 0.173
Train accuracy: 0.936
